# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll then also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

## Loading the Data

Start by loading in the data stored in the file **ToothGrowth.csv**.

In [2]:
# Your code here
import pandas as pd
df = pd.DataFrame(pd.read_csv('ToothGrowth.csv'))

In [4]:
df.sample(10)

,len,supp,dose
42,23.6,OJ,1.0
37,9.4,OJ,0.5
40,19.7,OJ,1.0
50,25.5,OJ,2.0
5,10.0,VC,0.5
9,7.0,VC,0.5
24,26.4,VC,2.0
23,25.5,VC,2.0
51,26.4,OJ,2.0
45,25.2,OJ,1.0


## Generating the ANOVA Table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage 

In [5]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [79]:
df.dose.value_counts()

2.0    20
1.0    20
0.5    20
Name: dose, dtype: int64

In [80]:
# dose is categorical!
formula = 'len ~ C(supp) + C(dose)'

In [81]:
lm = ols(formula, df).fit()

In [82]:
table = sm.stats.anova_lm(lm, typ=2)

In [83]:
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,14.016638,4.292793e-04
C(dose),2426.434333,2.0,82.810935,1.871163e-17
Residual,820.425000,56.0,NaN,NaN


## Reading the Table

Make a brief comment regarding the statistics regarding the effect of supplement and dosage on tooth length.

Both doseage and supplement seem to explain tooth length, with doseage being more significant

## Comparing to T-Tests

Now that you've gotten a brief chance to interact with ANOVA, its interesting to compare the results to those from the t-tests you were just working with. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterwards, you'll conduct a t-test to compare the tooth length of these two different samples.

In [84]:
import scipy.stats as stats
import numpy as np

In [85]:
df_oj = df[df['supp']=='OJ']
df_vc = df[df['supp']=='VC']
print(f"1. Sample Size: {len(df_oj)}, Variance btwn Supp and Length {np.var(df_oj['len'], ddof=1)}")

print(f"2. Sample Size: {len(df_vc)}, Variance btwn Supp and Length {np.var(df_vc['len'], ddof=1)}")

1. Sample Size: 30, Variance btwn Supp and Length 43.6334367816092
2. Sample Size: 30, Variance btwn Supp and Length 68.32722988505746


Now compare a t-test between these two groups and print the associated two-sided p-value.

In [86]:
#Your code here; calculate the 2-sided p-value for a t-test comparing the two supplement groups.
stats.ttest_ind(df_oj['len'], df_vc['len'])

Ttest_indResult(statistic=1.91526826869527, pvalue=0.06039337122412849)

## A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [87]:
#Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying 
#implementations)

formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)

In [88]:
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,3.668253,0.060393
Residual,3246.859333,58.0,NaN,NaN


## Generating Multiple T-Tests

While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [89]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


In [90]:
type(group_name[1])

numpy.float64

While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2 sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [91]:
#Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)
pairs = []
import itertools
for group in df.groupby(['supp', 'dose'])['len']:
    pairs.append(group[0])
ttest_groups = list(itertools.combinations(pairs, 2))
p_values = []
for group in ttest_groups:
    df_1 = df[(df['supp'] == group[0][0]) & (df['dose'] == group[0][1])]
    df_2 = df[(df['supp'] == group[1][0]) & (df['dose'] == group[1][1])]
    p = stats.ttest_ind(df_1['len'], df_2['len'], equal_var = False)[1]
    p_values.append(p)
p_value_dict = dict(zip(ttest_groups, p_values))
print(f"There are {len(ttest_groups)} combinations of supplement and dosage values\non which we can calculate independent t-tests.")
print(f"\nThe p-values are as shown below:")
for item, value in p_value_dict.items():
    print(item, value)

There are 15 combinations of supplement and dosage values
on which we can calculate independent t-tests.

The p-values are as shown below:
(('OJ', 0.5), ('OJ', 1.0)) 8.784919055161479e-05
(('OJ', 0.5), ('OJ', 2.0)) 1.3237838776972294e-06
(('OJ', 0.5), ('VC', 0.5)) 0.006358606764096813
(('OJ', 0.5), ('VC', 1.0)) 0.04601033257637553
(('OJ', 0.5), ('VC', 2.0)) 7.196253524006043e-06
(('OJ', 1.0), ('OJ', 2.0)) 0.039195142046244004
(('OJ', 1.0), ('VC', 0.5)) 3.6552067303259103e-08
(('OJ', 1.0), ('VC', 1.0)) 0.001038375872299884
(('OJ', 1.0), ('VC', 2.0)) 0.09652612338267014
(('OJ', 2.0), ('VC', 0.5)) 1.3621396478988818e-11
(('OJ', 2.0), ('VC', 1.0)) 2.3610742020468435e-07
(('OJ', 2.0), ('VC', 2.0)) 0.9638515887233756
(('VC', 0.5), ('VC', 1.0)) 6.811017702865016e-07
(('VC', 0.5), ('VC', 2.0)) 4.6815774144921145e-08
(('VC', 1.0), ('VC', 2.0)) 9.155603056638692e-05


## Summary

In this lesson, you examined the ANOVA technique to generalize A/B testing methods to multiple groups and factors.